In [16]:
import pandas as pd
import matplotlib.pyplot as plt
from pylab import *
import numpy as np
import rasterio
import pyproj

In [17]:
df = pd.read_csv('Subset_remote sensing data.csv')

In [18]:
df.head()

,productId,entityId,acquisitionDate,cloudCover,processingLevel,path,row,min_lat,min_lon,max_lat,max_lon,download_url
0,LC08_L1TP_040037_20140525_20170306_01_T1,LC80400372014145LGN01,22:04.5,21.26,L1TP,40,37,32.10775,-118.40269,34.22325,-115.92415,https://s3-us-west-2.amazonaws.com/landsat-pds...
1,LC08_L1TP_040037_20140610_20170305_01_T1,LC80400372014161LGN01,22:14.4,42.31,L1TP,40,37,32.10761,-118.41571,34.22310,-115.93687,https://s3-us-west-2.amazonaws.com/landsat-pds...
2,LC08_L1TP_040037_20140626_20170304_01_T1,LC80400372014177LGN01,22:15.6,75.72,L1TP,40,37,32.10785,-118.39292,34.22337,-115.91779,https://s3-us-west-2.amazonaws.com/landsat-pds...


In [19]:
#raster processing reference - https://geohackweek.github.io/raster/04-workingwithrasters/
def getband(url, bandfile):
   
    with rasterio.open(url+bandfile) as src:
        profile = src.profile
        oviews = src.overviews(1) # list of overviews from biggest to smallest
        oview = oviews[1]  # Use second-highest resolution overview
        #print('Decimation factor= {}'.format(oview))
        band = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))
        band = band.astype('f4')
        #band[band==0] = np.nan
        band[band==0] = -2
    return band, oview

In [20]:
def saveband(url, bandfile, localname, save=1):
    band, oview = getband(url, bandfile)
    
    with rasterio.open(url+bandfile) as src:
        profile = src.profile.copy()

        aff = src.transform
        newaff = rasterio.Affine(aff.a * oview, aff.b, aff.c,
                             aff.d, aff.e * oview, aff.f)
        profile.update({
            'dtype': 'float32',
            'height': band.shape[0],
            'width': band.shape[1],
            'transform': newaff})
             
    with rasterio.open(localname, 'w', **profile) as dst:
            dst.write_band(1, band)
    return band

In [23]:
for item in df.itertuples():
    #print(item)
    
    url = item.download_url.split("/index.html")[0]
    filename = url.split("/")[-1]
    url = url+"/"
#     blueband = filename+'_B{}.TIF'.format(2)
#     greenband = filename+'_B{}.TIF'.format(3)
    redband = filename+'_B{}.TIF'.format(4)
#     nirband = filename+'_B{}.TIF'.format(5)
    swir2band = filename+'_B{}.TIF'.format(7)
    thermalband = filename+'_B{}.TIF'.format(10)
    
    try:
#         localname = 'tiff images/blue/'+blueband
#         blue = saveband(url, blueband, localname)
        
#         localname = 'tiff images/green/'+greenband
#         green = saveband(url, greenband, localname)
        
        localname = 'tiff images/red/'+redband
        red = saveband(url, redband, localname)
    
#         localname = 'tiff images/nir/'+nirband
#         nir = saveband(url, nirband, localname)
    
        localname = 'tiff images/swir2/'+swir2band
        swir2 = saveband(url, swir2band, localname)
        
        localname = 'tiff images/thermal/'+thermalband
        thermal = saveband(url, thermalband, localname)
        
    except Exception as e:
        print(sys.exc_info()[0],"occured.")
        print(e)   

In [24]:
from skimage import  img_as_ubyte
from skimage import exposure
from pathlib import Path

i = 0
for item in df.itertuples():
    filename = df['productId'][i]
    
    blueband = 'tiff images/blue/'+filename+'_B{}.TIF'.format(2)
#     greenband = 'tiff images/green/'+filename+'_B{}.TIF'.format(3)
    redband = 'tiff images/red/'+filename+'_B{}.TIF'.format(4)
#     nirband = 'tiff images/nir/'+filename+'_B{}.TIF'.format(5)
    swir2band = 'tiff images/swir2/'+filename+'_B{}.TIF'.format(7)
    thermalband = 'tiff images/thermal/'+filename+'_B{}.TIF'.format(10)
    
#     with rasterio.open(blueband) as blueband_raster:
# #         blueband_data = blueband_raster.read(1)
#         transform=blueband_raster.transform
#         crs=blueband_raster.crs
#         profile=blueband_raster.profile
#     with rasterio.open(greenband) as greenband_raster:
#         greenband_data = greenband_raster.read(1)
    with rasterio.open(redband) as redband_raster:
        redband_data = redband_raster.read(1)
        transform=redband_raster.transform
        crs=redband_raster.crs
        profile=redband_raster.profile
#     with rasterio.open(nirband) as nirband_raster:
#         nirband_data = nirband_raster.read(1)
    with rasterio.open(swir2band) as swir2band_raster:
        swir2band_data = swir2band_raster.read(1)
    with rasterio.open(thermalband) as thermalband_raster:
        thermalband_data = thermalband_raster.read(1)
    channels=np.empty([3,redband_data.shape[0],redband_data.shape[1]],dtype=np.uint8)
    
    for index,image in enumerate([redband_data,swir2band_data,thermalband_data]):
        stretched=exposure.equalize_hist(image)    
        channels[index,:,:] = img_as_ubyte(stretched)
        
    filename = filename+'.tiff'
    #print(filename)
    tif_filename = 'tiff images'/ Path(filename)
    num_chans, height, width = channels.shape
    with rasterio.open(tif_filename,'w',driver='GTiff',
                       height=height,width=width,
                       count=num_chans,dtype=channels.dtype,
                       crs=crs,transform=transform, nodata=0.0) as dst:
        dst.write(channels)
        keys=['4','7','10']
        for index,chan_name in enumerate(keys):
            dst.update_tags(index+1,name=chan_name)
            
            
    with rasterio.open(tif_filename) as infile:
        #print(f"\nnew profile: {pprint.pformat(infile.profile)}\n")
        profile=infile.profile
        #
        # change the driver name from GTiff to JPEG
        #
        profile['driver']='JPEG'
        #
        # pathlib makes it easy to add a new suffix to a
        # filename
        raster=infile.read()
        jpeg_filename=tif_filename.with_suffix('.jpeg')
        with rasterio.open(jpeg_filename, 'w', **profile) as dst:
            dst.write(raster)
    
    i += 1

/Users/shalu/anaconda3/lib/python3.7/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
